In [1]:
import os
os.chdir('..')

In [2]:
data_folder = '../data/'
server_folder = './server/'
template_folder = './templates/'

import numpy as np
import json

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
directions = ['grad', 'hess', 'bfgs', 'bfgs-inv', 'hybrid', 'hybrid-inv']
types = ['LS', 'LS-ABS', 'TR', 'TR-ABS']

unfeasible = ['TR_grad', 'TR-ABS_grad', 'LS_hybrid', 'TR_hybrid', 'TR_bfgs-inv', 'TR-ABS_bfgs-inv', 'LS_hybrid-inv', 'TR_hybrid-inv', 'TR-ABS_hybrid-inv']

combinations = []

# Add all combinations
for d in directions:
    for t in types:
        str_ = t + '_' + d
        
        if str_ not in unfeasible:
            combinations.append(str_)

In [4]:
models1 = ['LPMC_DC_S', 
          'LPMC_DC_M', 
          'LPMC_DC_L',
          'LPMC_RR_S',
          'LPMC_RR_M',
          'LPMC_RR_L']
models2 = ['LPMC_Full_S',
          'LPMC_Full_M',
          'LPMC_Full_L',
          'MTMC']

models = ['LPMC_DC_S', 
          'LPMC_DC_M', 
          'LPMC_DC_L',
          'LPMC_RR_S',
          'LPMC_RR_M',
          'LPMC_RR_L',
          'LPMC_Full_S',
          'LPMC_Full_M',
          'LPMC_Full_L',
          'MTMC']

In [5]:
order = ['LS_grad',
         'LS_bfgs',
         'LS_bfgs-inv',
         'TR_bfgs',
         'LS_hess',
         'TR_hess',
         'LS-ABS_grad',
         'LS-ABS_bfgs',
         'LS-ABS_bfgs-inv',
         'TR-ABS_bfgs',
         'LS-ABS_hess',
         'TR-ABS_hess',
         'LS-ABS_hybrid',
         'TR-ABS_hybrid',
         'LS-ABS_hybrid-inv']

In [6]:
comb_to_str = {
    'LS_grad': 'GD',
    'LS_bfgs': 'BFGS',
    'LS_bfgs-inv': 'BFGS$^{-1}$',
    'TR_bfgs': 'TR-BFGS',
    'LS_hess': 'NM',
    'TR_hess': 'TR',
    'LS-ABS_grad': 'GD-ABS',
    'LS-ABS_bfgs': 'BFGS-ABS',
    'LS-ABS_bfgs-inv': 'BFGS$^{-1}$-ABS',
    'TR-ABS_bfgs': 'TR-BFGS-ABS',
    'LS-ABS_hess': 'NM-ABS',
    'TR-ABS_hess': 'TR-ABS',
    'LS-ABS_hybrid': 'H-NM-ABS',
    'TR-ABS_hybrid': 'H-TR-ABS',
    'LS-ABS_hybrid-inv': 'HAMABS'
}

In [7]:
max_epochs = 1000

# Full tables

In [8]:
results = {}

for mod in models:
    results[mod] = {}
    
    fastest_time = {'name': None, 'avg': 99999999}
    fastest_epochs = {'name': None, 'avg': 99999999}
    
    for comb in combinations:
        
        tmp = {'time': {'avg': 99999, 'std': 999},
               'epochs': {'avg': 99999, 'std': 999, 'stop': False}}                
        try:
            with open(server_folder + mod + '/results/' + comb + '.json', 'r') as infile:
                data = json.load(infile)

            tmp['time']['avg'] = np.mean(data['time'])
            tmp['time']['std'] = np.std(data['time'])
            tmp['epochs']['avg'] = np.mean(data['epochs'])
            tmp['epochs']['std'] = np.std(data['epochs'])
            
            if tmp['epochs']['avg'] >= max_epochs:
                tmp['epochs']['stop'] = 'True'
            
            if np.mean(data['time']) < fastest_time['avg'] and tmp['epochs']['avg'] < max_epochs:
                fastest_time['name'] = comb
                fastest_time['avg'] = np.mean(data['time'])
                
            if np.mean(data['epochs']) < fastest_epochs['avg']:
                fastest_epochs['name'] = comb
                fastest_epochs['avg'] = np.mean(data['epochs'])
            
            results[mod][comb] = tmp
                    
        except FileNotFoundError:
            pass
        
    results[mod]['fastest'] = {'time': fastest_time['name'], 'epochs': fastest_epochs['name']}

In [9]:
for s in ['time', 'epochs']:

    for i, mod in enumerate([models1, models2]):

        print('\\begin{landscape}')
        print('\\vspace*{\\fill}')
        print('\\begin{table}[H]')        
        print('  \\renewcommand\\arraystretch{1.1}')
        print('  \centering')
        if i == 0:
            print('  \\begin{tabular}{l||c|c|c|c|c|c}')
            print('    \multirow{2}{*}{Algorithms} & \multicolumn{6}{c}{Models} \\\\ \cline{2-7}')
        else:
            print('  \\begin{tabular}{l||c|c|c||c}')
            print('    \multirow{2}{*}{Algorithms} & \multicolumn{4}{c}{Models} \\\\ \cline{2-5}')

        str_models = ['\\texttt{' + x.replace('_', '\_') + '}' for x in mod]
        print('  & {} \\\\ \hline\hline'.format(' & '.join(str_models)))

        for alg in order:

            res_models = []
            for m in mod:
                tmp = results[m][alg][s]

                pre = ''
                if results[m]['fastest'][s] == alg:
                    pre = '\\cellcolor{black!15}\\boldmath'

                if results[m][alg]['epochs']['stop']:
                    pre = '\color{gray}'
                    
                if s == 'epochs' and 'ABS' not in comb_to_str[alg]:
                    res_models.append(pre+'${:.0f}$'.format(tmp['avg']))
                else:
                    res_models.append(pre+'${:.2f} \pm {:.2f}$'.format(tmp['avg'], tmp['std']))
            
            tmp = '\\\\'
            
            if comb_to_str[alg] != 'HAMABS':
                tmp = tmp + '\hline'
            
            if comb_to_str[alg] == 'TR' or comb_to_str[alg] == 'TR-ABS':
                tmp = tmp + '\hline'

            print('    \\texttt{' + comb_to_str[alg] + '} & ' + ' & '.join(res_models) + ' ' + tmp)
        

        print('  \end{tabular}')
        print('  \caption{CAPTION => ' + s + ' }')
        print('  \label{tab:results_' + s + '}')
        print('\end{table}')
        print('\\vspace*{\\fill}')
        print('\end{landscape}')
        print('\n')

\begin{landscape}
\vspace*{\fill}
\begin{table}[H]
  \renewcommand\arraystretch{1.1}
  \centering
  \begin{tabular}{l||c|c|c|c|c|c}
    \multirow{2}{*}{Algorithms} & \multicolumn{6}{c}{Models} \\ \cline{2-7}
  & \texttt{LPMC\_DC\_S} & \texttt{LPMC\_DC\_M} & \texttt{LPMC\_DC\_L} & \texttt{LPMC\_RR\_S} & \texttt{LPMC\_RR\_M} & \texttt{LPMC\_RR\_L} \\ \hline\hline
    \texttt{GD} & \color{gray}$52.16 \pm 0.24$ & \color{gray}$95.07 \pm 0.45$ & \color{gray}$140.95 \pm 0.49$ & \color{gray}$303.21 \pm 0.25$ & \color{gray}$567.61 \pm 0.45$ & \color{gray}$820.38 \pm 1.46$ \\\hline
    \texttt{BFGS} & $6.78 \pm 0.03$ & $13.11 \pm 0.06$ & $17.96 \pm 0.06$ & $183.83 \pm 0.50$ & $337.25 \pm 0.30$ & $492.90 \pm 0.39$ \\\hline
    \texttt{BFGS$^{-1}$} & $7.20 \pm 0.02$ & $12.96 \pm 0.10$ & $19.61 \pm 0.12$ & $177.66 \pm 0.25$ & $332.93 \pm 0.40$ & $473.32 \pm 0.98$ \\\hline
    \texttt{TR-BFGS} & $5.40 \pm 0.04$ & $11.05 \pm 0.11$ & $17.17 \pm 0.07$ & $227.83 \pm 0.51$ & $405.82 \pm 0.60$ & \color{gr

# HAMABS against Biogeme

In [10]:
results = {}

for mod in models:    
    results[mod] = {}
    
    times = {'avg': 99999, 'std': 999}
    epochs = {'avg': 99999, 'std': 999}

    try:
        with open(server_folder + mod + '/results/biogeme_bfgs.json', 'r') as infile:
            data = json.load(infile)

        avg_time = np.mean(data['time'])
        avg_epochs = np.mean(data['epochs'])

        times['bio_avg'] = np.mean(data['time'])
        times['bio_std'] = np.std(data['time'])
        epochs['bio_avg'] = np.mean(data['epochs'])
        epochs['bio_std'] = np.std(data['epochs'])
        
    except FileNotFoundError:
        
        times['bio_avg'] = 100
        times['bio_std'] = 9

        epochs['bio_avg'] = 100
        epochs['bio_std'] = 9
        
    try:
        with open(server_folder + mod + '/results/LS-ABS_hybrid-inv.json', 'r') as infile:
            data = json.load(infile)

        times['avg'] = np.mean(data['time'])
        times['std'] = np.std(data['time'])
        epochs['avg'] = np.mean(data['epochs'])
        epochs['std'] = np.std(data['epochs'])

    except FileNotFoundError:
        pass
        
    if times['avg'] > times['bio_avg']:
        times['ratio'] = "$\\times {:.2f}$".format(times['avg']/times['bio_avg'])
    else:
        times['ratio'] = "$\div {:.2f}$".format(times['bio_avg']/times['avg'])
        
        
    if epochs['avg'] > epochs['bio_avg']:
        epochs['ratio'] = "$\\times {:.2f}$".format(epochs['avg']/epochs['bio_avg'])
    else:
        epochs['ratio'] = "$\div {:.2f}$".format(epochs['bio_avg']/epochs['avg'])        
        
    results[mod]['time'] = times
    results[mod]['epochs'] = epochs

In [11]:
for s in ['time', 'epochs']:
    print('Make table for {}\n'.format(s))

    print('\\begin{table}[H]')
    print('  \centering')
    print('  \\renewcommand\\arraystretch{1.1}')
    print('  \\begin{tabular}{l||c|c|c}')
    
    if s == 'time':
        name = 'Time [s]'
    else:
        name = 'Epochs'
    print('    \multirow{2}{*}{Models} & \multicolumn{2}{c|}{' + name + '} & \multirow{2}{*}{Speedup} \\\\ \cline{2-3}')
              
    print('    & \\texttt{HAMABS} & Biogeme/Scipy & \\\\ \hline \hline')

    for mod in models:

        res = results[mod][s]
        
        prec = ':.2f'

        val_res = ''
        if res['std'] == 0:
            val_res = '${:d}$'.format(int(res['avg']))
        else:
            str_res = '${{{}}} \pm {{{}}}$'.format(prec, prec)
            val_res = str_res.format(res['avg'], res['std'])

        val_bio = ''
        if res['bio_std'] == 0:
            val_bio = '${:d}$'.format(int(res['bio_avg']))
        else:
            str_res = '${{{}}} \pm {{{}}}$'.format(prec, prec)
            val_bio = str_res.format(res['bio_avg'], res['bio_std'])
            
        end = ' \\\\'
        
        if '_L' in mod:
            end = end + ' \hline'
            
        if mod == 'LPMC_Full_L':
            end = end + ' \hline'
        
        print('    \\texttt{{{}}} & '.format(mod.replace('_', '\_')) + val_res + ' & ' + val_bio + ' & ' + res['ratio'] + end)

    print('  \end{tabular}')
    print('  \caption{CAPTION => ' + s + ' }')
    print('  \label{tab:comp_biogeme_' + s + '}')
    print('\end{table}')
    print('\n')

Make table for time

\begin{table}[H]
  \centering
  \renewcommand\arraystretch{1.1}
  \begin{tabular}{l||c|c|c}
    \multirow{2}{*}{Models} & \multicolumn{2}{c|}{Time [s]} & \multirow{2}{*}{Speedup} \\ \cline{2-3}
    & \texttt{HAMABS} & Biogeme/Scipy & \\ \hline \hline
    \texttt{LPMC\_DC\_S} & $1.86 \pm 0.12$ & $1.62 \pm 0.01$ & $\times 1.15$ \\
    \texttt{LPMC\_DC\_M} & $3.11 \pm 0.20$ & $2.79 \pm 0.02$ & $\times 1.11$ \\
    \texttt{LPMC\_DC\_L} & $4.59 \pm 0.32$ & $4.07 \pm 0.04$ & $\times 1.13$ \\ \hline
    \texttt{LPMC\_RR\_S} & $11.98 \pm 1.23$ & $65.17 \pm 0.09$ & $\div 5.44$ \\
    \texttt{LPMC\_RR\_M} & $18.46 \pm 1.06$ & $127.67 \pm 0.30$ & $\div 6.91$ \\
    \texttt{LPMC\_RR\_L} & $18.14 \pm 1.06$ & $177.09 \pm 0.29$ & $\div 9.76$ \\ \hline
    \texttt{LPMC\_Full\_S} & $257.02 \pm 42.85$ & $1462.51 \pm 14.41$ & $\div 5.69$ \\
    \texttt{LPMC\_Full\_M} & $405.43 \pm 43.77$ & $2480.06 \pm 18.27$ & $\div 6.12$ \\
    \texttt{LPMC\_Full\_L} & $486.31 \pm 63.38$ & $4758.28